# Поиск наиболее похожих товаров

## Введение

Заказчик предоставил анонимизированный набор имеющихся на складах товаров и набор новых товаров, для которых необходимо найти наиболее похожие из имеющегося набора. 
Особенность задачи состоит в том, что автоматический подбор похожих товаров должен производиться так, чтобы симулировать работу экспертов. Образцы их ответов предоставлены для обучения модели в отдельном файле вместе с примерами запросов.

## Описание данных

- <u>base.csv</u> - анонимизированный **набор товаров**. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.

- <u>train.csv</u> - **обучающий датасет**. Каждая строчка - один товар, для которого известен уникальный id (0-query, …, 100000-query), вектор признаков и id товара из base.csv, который максимально похож на него по мнению экспертов.

- <u>validation.csv</u> - **датасет с товарами** (уникальный id и вектор признаков), **для которых надо найти наиболее близкие** товары из base.csv.

- <u>validation_answer.csv</u> - **правильные ответы** к датасету с товарами для поиска (даётся **для оценки** работы простроенной **модели**).


## Задачи

- **разработать алгоритм**, который для всех товаров из validation.csv предложит несколько вариантов наиболее похожих товаров из base.csv;
- **оценить качество алгоритма** по метрике **accuracy@5**.

## Вывод

1. В рамках исследования были представлены 3 набора данных:
- Пропусков и дубликатов нет ни в одном наборе.
- Данные в основном представлены непрерывными числовыми значениями, основная их доля очевидно имеет нормальное распределение. 
- Данные, которые явно не характеризуются нормальным распределением были исключены из анализа, т.к. такие данные вносят лишние шумы.
- Сильных корреляционных зависимостей не выявлено.

2. Учитывая тот факт, что формула для расчёта расстояния между точками в пространстве чувствительна к диапазону значений различных характеристик, данные были нормализованы.

3. Дополнительно проанализирована целевая метрика в зависимости от количества кандидатов в диапозоне от 5 до 500 кандадатов и установлено:
- с увеличением количества кандидатов растет и максимальное значение целевой метрики
- увеличение количества кандидатов увеличит и время предсказания модели, так как ей придется в каждой иттерации просчитывать бОльшее количество кандидатов.

4. Для решения поставленной задачи было создано и наполнено векторное пространство из объектов анонамизированного набора данных (base). Пространство поделено на 20 кластеров, что ускоряет и уточняет процесс поиска. Для каждого объекта из train были найдены 10 ближайших векторов при помощи меры FlatL2 (при большом количестве не хватало оперативной памяти при формировании дататсетов для обучения и тестирования ML модели).

5. На тренировочном датасете (train) получаем значение метрики **accuracy@5=63.114**, что является удовлетворительным результатом работы модели. Такой результат удалось достичь за счет удалениея не информативных признаков и стандартизации данных. При этом на тестовой выборке (validation) значение метрики **accuracy@5= 62.96** достигнуто без учета ранжирования ML моделью.

6. В качестве ML модели был выбран CatBoostClassifier и проведен подбор гиперпараметров. По обученной модели пердсказаны вероятности совместимости и по найденным занчениям вероятностей  определены 5 наиболее подходящих кандидато по которым расчитано значение целевой метрики **accuracy@5=61.81**

**На текущем этапе мы видим, что применение ML модели не улучшает показатель целевой метрики, тем не менее в дальнейшем можно поробовать усовершенстовать модель и повторно сравнить результат**

## Импорт библиотек

In [2]:
!pip install faiss-cpu --no-cache

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import faiss
import optuna
import datetime as dt

#from adjdatatools.preprocessing import AdjustedScaler
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer, f1_score, roc_curve
from optuna.samplers import TPESampler
from catboost import CatBoostClassifier
RANDOM_STATE = 12345

# Загрузка и анализ данных

In [6]:
# функция для предварительного знакомства с данными
def first_look(df, num_of_srtings=5):
    print('Общая информация')
    display(df.info())
    
    print(f'Первые {num_of_srtings} строк(и) данных')
    display(df.head(num_of_srtings))
    
    print('Основные статистические характеристики данных')
    display(df.describe())
    
    print('Количество пропусков:')
    print(df.isna().sum())
    print()
    
    print('Количество дубликатов:', df.duplicated().sum())

In [7]:
# открываем файл с анонимизированным набором товаров 
df_base = pd.read_csv("base.csv", index_col=0)
df_train = pd.read_csv("train.csv", index_col=0)
df_validation = pd.read_csv("validation.csv", index_col=0)
df_validation_answer = pd.read_csv("validation_answer.csv", index_col=0)

## Анонимизированный набор товаров

In [8]:
first_look(df_base)

Общая информация
<class 'pandas.core.frame.DataFrame'>
Index: 2918139 entries, 0-base to 4744766-base
Data columns (total 72 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      float64
 17  17      float64
 18  18      float64
 19  19      float64
 20  20      float64
 21  21      float64
 22  22      float64
 23  23      float64
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      float64
 32  32      float64
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39 

None

Первые 5 строк(и) данных


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


Основные статистические характеристики данных


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,...,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06
mean,-8.622947e+01,8.080077e+00,-4.458080e+01,-1.466350e+02,1.113166e+02,-7.199138e+01,-3.922239e+02,2.035283e+01,1.236842e+02,1.244581e+02,...,-7.902286e+01,3.329735e+01,-1.547962e+02,1.415132e+01,6.779167e+01,2.354490e+01,7.495930e+01,1.155667e+02,-7.993390e+02,-4.779125e+01
std,2.489132e+01,4.953387e+00,3.863166e+01,1.984480e+01,4.634809e+01,2.818607e+01,2.716550e+02,6.421638e+01,6.356109e+00,6.443058e+01,...,3.045642e+01,2.888603e+01,4.122929e+01,9.895115e+01,1.823356e+00,5.534224e+01,6.134500e+01,2.117518e+01,3.854131e+02,4.174802e+01
min,-1.994687e+02,-1.391461e+01,-2.400734e+02,-2.326671e+02,-1.055830e+02,-2.110086e+02,-7.914699e+02,-3.018597e+02,9.315305e+01,-1.738719e+02,...,-2.205662e+02,-8.850774e+01,-3.539028e+02,-1.575944e+02,5.950944e+01,-2.331382e+02,-2.036016e+02,1.572448e+01,-1.297931e+03,-2.267801e+02
25%,-1.030654e+02,4.708491e+00,-6.955949e+01,-1.599051e+02,8.050795e+01,-9.137994e+01,-6.293318e+02,-2.222147e+01,1.194840e+02,8.176751e+01,...,-9.876390e+01,1.698862e+01,-1.807799e+02,-7.130038e+01,6.658096e+01,-1.251624e+01,3.377574e+01,1.016867e+02,-1.074465e+03,-7.566641e+01
50%,-8.623150e+01,8.038950e+00,-4.381661e+01,-1.467768e+02,1.118730e+02,-7.192230e+01,-4.222016e+02,2.080477e+01,1.238923e+02,1.234977e+02,...,-7.848812e+01,3.471502e+01,-1.539773e+02,1.382693e+01,6.781458e+01,2.341649e+01,7.492997e+01,1.160244e+02,-1.074465e+03,-4.859196e+01
75%,-6.925658e+01,1.147007e+01,-1.962527e+01,-1.333277e+02,1.423743e+02,-5.244111e+01,-1.566686e+02,6.391821e+01,1.279705e+02,1.672206e+02,...,-5.853355e+01,5.216429e+01,-1.273405e+02,9.966753e+01,6.902666e+01,5.975511e+01,1.158760e+02,1.295524e+02,-5.057445e+02,-1.971424e+01
max,2.151555e+01,2.993721e+01,1.609372e+02,-5.137478e+01,3.196645e+02,5.880624e+01,1.096325e+02,3.412282e+02,1.522612e+02,4.275421e+02,...,6.017411e+01,1.541678e+02,2.436099e+01,1.850981e+02,7.571203e+01,3.148988e+02,3.395738e+02,2.147063e+02,9.877081e+01,1.269732e+02


Количество пропусков:
0     0
1     0
2     0
3     0
4     0
     ..
67    0
68    0
69    0
70    0
71    0
Length: 72, dtype: int64

Количество дубликатов: 0


Посмотрим на распределение значений признаков.

In [ ]:
df_base.hist(figsize=(18, 16), bins=50, grid=False);
plt.tight_layout()
plt.show()

In [ ]:
df_base = df_base.drop(['6', '21', '25', '33', '44', '59', '65', '70'], axis=1)

**Вывод**
- Набор товаров, среди которых нам нужно проводить поиск содержит почти 3 млн образцов
- Данные зашифрованы. Для каждого образца товара имеется 72 характеристики все представляют собой дробные числа
- Пропусков и дубликатов в наборе нет
- Как видно из гистограмм, большинство признаков имеют бликое к нормальному распределению. Но имеются признаки никак не связанные с нормальным распределением. В первой иттерации удалим эти признаки.

## Данные, для которых даны оценки экспертов

In [ ]:
first_look(df_train)

Посмотрим на распределение значений признаков.

In [ ]:
df_train.hist(figsize=(18, 16), bins=100, grid=False);
plt.tight_layout()
plt.show()

**Вывод**
- Данные, для которых даны оценки экспертов без пропусков и дубликатов
- Характеристики имеют те же распределения, что и в базовом датасете. Также удалим признаки
- Оценки экспертов содержатся в отдельном столбце target. Выделим его в отдельную переменную для последующей передачи модели.

In [ ]:
# разделим запросы и ответы экспертов
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

In [ ]:
df_train = df_train.drop(['6', '21', '25', '33', '44', '59', '65', '70'], axis=1)

## Данные для проверки качества модели

In [ ]:
first_look(df_validation)

Посмотрим на распределение значений признаков.

In [ ]:
df_validation.hist(figsize=(18, 16), bins=100, grid=False);
plt.tight_layout()
plt.show()

**Выводы** для датасета с запросами, для которых необходимо подобрать похожие товары в соответствии с рекомендациями экспертов аналогичны предидущему датасету.

In [ ]:
df_validation = df_validation.drop(['6', '21', '25', '33', '44', '59', '65', '70'], axis=1)

## Корреляция на базе анонимизированного набора товаров

In [ ]:
f, ax = plt.subplots(figsize=(18, 18))
sns.heatmap(df_base.corr(), vmin=-1, vmax=1, square=True, annot=False, fmt='.2f');
plt.title('Матрица коэффициентов корреляции Пирсона')
plt.show()

## Вывод после предварительного знакомства с данными

Для исследования доступны три набора данных
- 3 млн товаров, среди которых мы должны подбирать похожие на входные запросы
- 100 000 строк запросов с ответами экспертов
- 100 000 запросов для подбора похожих товаров с помощью разрабатываемой модели.

Все три набора данных зашифрованы Для каждого образца товара имеется 72 характеристики все представляют собой дробные числа. 

Пропусков и дубликатов нет ни в одном наборе.

Данные в основном представлены непрерывными числовыми значениями, основная их доля очевидно имеет нормальное распределение. Те данные, которые явно не характеризуются нормальным распределением, будут вносить лишний шум в результаты метчинга - такие данные исклчюены.

Сильных корреляционных зависимостей не наблюдается.


# Нормализация данных

Так как формула для расчёта расстояния между точками в пространстве чувствительна к диапазону значений различных характеристик, предварительно нормализуем данные.


In [ ]:
scaler = StandardScaler()
scaler.fit(df_base[df_base.columns])
df_base[df_base.columns]             = scaler.transform(df_base[df_base.columns])
df_train[df_train.columns]           = scaler.transform(df_train[df_train.columns])
df_validation[df_validation.columns] = scaler.transform(df_validation[df_validation.columns])

# Поиск ближайших соседей

Для того, чтобы найти в базовом наборе товары, похожие на наш запрос, воспользуемся библиотекой FAISS.

Принцип работы Faiss:
- Отображение всех объектов (векторов) в едином пространстве
- Деление полученного пространства на отдельные части (кластеры) при помощи спец. алгоритма библиотеки
- Для каждого кластера происходит поиск его центра (центроида)
- Таким образом, для нового вектора можно быстрой найти ближайших соседей, вычислив расстояние именно до центроидов (не перемножая новый вектор со всеми остальными векторами).
- После выявления ближайшего кластера, новый вектор перемножается векторами только из этого кластера.

FlatL2 - мера близости вектором L2-норма (евклидово расстояние). По этой причине большое значение имеет "нормализованы ли признаки?", поскольку в этой ситуации признак бóльшей величины будет играть бóльшую роль в предсказании. Но это не отображает достоверность его значимость. quantizer - квантизатор, который получает на вход вектора размерностью dims и рассчитывает расстояние между ними по евклидовой норме. idx_l2 - пространство векторов размерностью dims, разделяемое на n_cells кластеров с помощью quantizer.

In [ ]:
# инициализация индекса
dims = df_base.shape[1]
n_cells = 18
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [ ]:
# подготовка к поиску
idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))

# создание словаря для нахождения индекса товара в базовом наборе данных
base_index = {k: v for k, v in enumerate(df_base.index.to_list())}

Далее расчитана целевая метрика в зависимости от количества кандидатов. По заданию метрика считается для 5 кандидатов, но ml модель может немного улучшить метрику. Ислледуем диапозон от 5 до 450 кандадатов

In [ ]:
cand_number = []
cand_result = []

for i in range(5, 450, 50):
    r, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), i)
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] for r in el])

    cand_number.append(i)
    cand_result.append(100 * acc / len(idx))

In [ ]:
test_cand = pd.DataFrame({'num_candidate': cand_number, 'acc': cand_result})

In [ ]:
plt.figure(figsize=(18, 6))
plt.plot(test_cand.num_candidate, test_cand.acc, 'ro-')
plt.grid(True)
plt.title('Зависимость между целевой метрикой и количеством кандадатов')
plt.ylabel('Значение целевой метрики')
plt.xlabel('Количество кандадатов')
plt.show()

Как видно, с увеличением метрики растетм и максимальное значение целевой метрики. Но нужно понимать, что увеличение количества кандидатов увеличит и время предсказания модели, так как ей придется в каждой иттерации просчитывать бОльшее количество кандидатов. В данном случае необходимо найти баланс между количеством кандидатов и затрачиваемого времени. В первом приближении предлагается использовать 20 кандидатов - значение целевой метрики около 67%.

## Поиск по обучающей выборке

In [ ]:
K_NEIGHBORS = 10

Проводим сопоставление товаров библиотекой Faiss, для получения результата по метрике accuracy@5 задаем подбор 5 кандидатов.

In [ ]:
# измерим время поиска
start_time = dt.datetime.now().timestamp()
vecs, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), K_NEIGHBORS)
time_spent = dt.datetime.now().timestamp() - start_time
print(f'Время поиска: {time_spent // 60} минут и {time_spent % 60} секунд')

Считаем метрику на тренировочном датасете.

In [ ]:
acc_n = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc_n += int(target in [base_index[r] for r in el])
print(100 * acc_n / len(idx))

In [ ]:
acc_5 = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc_5 += int(target in [base_index[r] for r in el[:5]])
print(100 * acc_5 / len(idx))

На текущему этапе создано и наполнено векторное пространство из объектов датасета base. Пространство поделено на 20 кластеров, что ускоряет и уточняет процесс поиска. Для каждого объекта из train были найдены 10 ближайших векторов при помощи меры FlatL2. На тренировочном датасете получаем значение метрики **accuracy@5=63.114**, что является удовлетворительным результатом работы модели. Такой результат удалось достичь за счет удалениея не информативных признаков и стандартизации данных. Для 10 кандадатов значение метрики **accuracy@30=67.186**. Далее, среди найденных кандидатов необходимо выявить самые подходящие. Для этого их требуется отранжировать.

**p.s. свыше 10 кандидатов првоерить не удалось - ругается на нехватку оперативной памяти.**

# Ранжирование

## Формирование тренировочной выборки

Првым шагом необходимо составить новый датафрейм состоящий из двух половин:

Искомые векторы
Несколько кандидатов на сопоставление для каждого искомого вектора.
Последним столбцом для этого вектора станет matching содержащий правильный ответ: 1 - вектор подходит, 0 - вектор не подходим.

In [ ]:
# подготовка датафрейма с перечнем ID новых и старых объектов
idx_df = pd.DataFrame(dtype='float32', data=idx, index=df_train.index)
idx_df = pd.melt(idx_df.T)
idx_df.columns = ['id_query', 'candidate']
idx_df['id_candidate'] = [base_index[number] for number in idx_df['candidate'].values]
idx_df.drop('candidate', axis=1, inplace=True)

idx_df

Полученные колонки с id товаров будут использованы для объединения при помощи метода merge(). В качестве данных будут использоваться уже отмасштабированные признаки

In [ ]:
df_train_base = idx_df.merge(df_train, left_on='id_query', right_on='Id', how='left')
df_train_base = df_train_base.merge(df_base, left_on='id_candidate', right_on='Id', how='left', suffixes=('', '_base'))
df_train_base = df_train_base.merge(targets, left_on='id_query', right_on='Id', how='left')

df_train_base.head()

In [ ]:
# преобразование целевого признака для бинарной классификации
df_train_base['matching'] = (df_train_base['id_candidate'] == df_train_base['Target']).astype('int')
df_train_base.drop('Target', axis=1, inplace=True)

In [ ]:
# выделение признаков и таргета
y_train = df_train_base['matching']
X_train = df_train_base.drop(['id_query', 'id_candidate', 'matching'] , axis=1)

display(y_train.to_frame())
display(X_train)

В результате получилась выборка для обучения:
- 1 000 000  записей = 100 000 новых товаров (query) * 10 ближайших кандидатов для каждого.
- 128 признака = 64 признака "старых" товаров из base + 64 признака "новых" товаров из train.
- бинарный целевой признак.

## Поиск по тестовой выборке

In [ ]:
# поиск ближайших соседей для валидационной выборки
start_time = dt.datetime.now().timestamp()
valid_vec, valid_idx = idx_l2.search(np.ascontiguousarray(df_validation), K_NEIGHBORS)
time_spent = dt.datetime.now().timestamp() - start_time
print(f'Время поиска: {time_spent // 60} минут и {time_spent % 60} секунд')

In [ ]:
acc_n_valid = 0

for target, el in zip(df_validation_answer['Expected'].values.tolist(), valid_idx.tolist()):
    acc_n_valid += int(target in [base_index[r] for r in el])

print(100 * acc_n_valid / len(valid_idx))

In [ ]:
acc_5_valid = 0

for target, el in zip(df_validation_answer['Expected'].values.tolist(), valid_idx.tolist()):
    acc_5_valid += int(target in [base_index[r] for r in el[:5]])

print(100 * acc_5_valid / len(valid_idx))

Результата на тестовой выборке (без ранжирования ML моделью) составялет 62.96, что весьма неплохо. Темне-менее попробуем улчишть с применением ML.

## Формирование тестовой выборки

Аналогичную процедуру  как и для обучающей выборки необходимо выполнить для тестовой выборки.

In [ ]:
# подготовка датафрейма с перечнем ID новых и старых объектов
idx_df = pd.DataFrame(dtype='float32', data=valid_idx, index=df_validation.index)
idx_df = pd.melt(idx_df.T)
idx_df.columns = ['id_query', 'candidate']
idx_df['id_candidate'] = [base_index[number] for number in idx_df['candidate'].values]
idx_df.drop('candidate', axis=1, inplace=True)

display(idx_df)

In [ ]:
df_valid_base = idx_df.merge(df_validation, left_on='id_query', right_on='Id', how='left')
df_valid_base = df_valid_base.merge(df_base, left_on='id_candidate', right_on='Id', how='left', suffixes=('', '_base'))
df_valid_base = df_valid_base.merge(df_validation_answer, left_on='id_query', right_on='Id', how='left')

df_valid_base.head()

In [ ]:
# преобразование целевого признака для бинарной классификации
df_valid_base['matching'] = (df_valid_base['id_candidate'] == df_valid_base['Expected']).astype('int')
display(df_valid_base[['id_query', 'id_candidate', 'Expected', 'matching']])
df_valid_base.drop('Expected', axis=1, inplace=True)

In [ ]:
# выделение признаков и таргета
y_valid = df_valid_base['matching']
X_valid = df_valid_base.drop(['id_query', 'id_candidate', 'matching'] , axis=1)

display(y_valid.to_frame())
display(X_valid)

## Обучение ML-модели

In [ ]:
y_train.value_counts(normalize=True)

Обучать будем CatBoostClassifier. Для подготовки модели осталось получить лучшие гиперпараметры. Предлагается воспользоваться библиотекой Optuna.В качестве метрики выбрана F1, так как в отличии от accuracy дисбаланс классов на нее не влияет, а у нас наблюдается сильный дисбаланс.

In [ ]:
def objective(trial: optuna.Trial):
    params = {
            'iterations': trial.suggest_int('iterations', 100, 1000, step=50),
            'learning_rate': trial.suggest_float('learning_rate', 0.0001, 1, step=0.0001),
            'depth': trial.suggest_int('depth', 2, 10, step=1),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.001, 100, log=True)
    }
    
    cbc = CatBoostClassifier(**params, random_state=RANDOM_STATE, verbose=False)
    cv = KFold(n_splits=4)
    scorer = make_scorer(f1_score)
    
    scores = cross_val_score(
        cbc, 
        X_train, 
        y_train, 
        cv=cv, 
        scoring=scorer)

    return scores.mean()


study = optuna.create_study(study_name='CatBoostClassifier', 
                                direction='maximize', 
                                sampler=TPESampler(seed=RANDOM_STATE)
                                )
study.optimize(objective, n_trials=1, timeout=None, n_jobs=1)

In [ ]:
print('Best hyperparameters:', study.best_params)
print('Best F1-score:', study.best_value)

Оптимальные гиперпараметры найдены. Следующим шагом перейдем к обучению модели и предсказанию вероятности совместимости. Затем по найденным занчениям вероятностей буду определены 5 наиболее подходящих кандидатов.

In [ ]:
model = CatBoostClassifier(iterations=950, learning_rate=0.3164,depth=3,l2_leaf_reg=0.010539048248947627,random_state=RANDOM_STATE, verbose=False)
model.fit(X_train, y_train)

In [ ]:
valid_probabilities = model.predict_proba(X_valid)[:, 1]
valid_proba_series = pd.Series(valid_probabilities)

In [ ]:
valid_candidates = []
for i in range(0, len(valid_proba_series), K_NEIGHBORS):
    query = valid_proba_series[i : i + K_NEIGHBORS]
    index = query.sort_values(ascending=False)[0:5].index
    valid_candidates.append(index)

print('Кол-во предсказаний:', len(valid_candidates))
print('Кол-во кандидатов:', len(valid_candidates[0]))

Теперь расчитаем метрику для 5 найденных наиболее подходящих кандидатов.

In [ ]:
acc_5_final = 0
for target, candidates in zip(df_validation_answer['Expected'].values.tolist(), valid_candidates):
    acc_5_final += int(target in df_valid_base.loc[candidates, 'id_candidate'].values)

print(100 * acc_5_final / len(valid_candidates))

# Вывод

1. В рамках исследования были представлены 3 набора данных:
- Пропусков и дубликатов нет ни в одном наборе.
- Данные в основном представлены непрерывными числовыми значениями, основная их доля очевидно имеет нормальное распределение. 
- Данные, которые явно не характеризуются нормальным распределением были исключены из анализа, т.к. такие данные вносят лишние шумы.
- Сильных корреляционных зависимостей не выявлено.

2. Учитывая тот факт, что формула для расчёта расстояния между точками в пространстве чувствительна к диапазону значений различных характеристик, данные были нормализованы.

3. Дополнительно проанализирована целевая метрика в зависимости от количества кандидатов в диапозоне от 5 до 500 кандадатов и установлено:
- с увеличением количества кандидатов растет и максимальное значение целевой метрики
- увеличение количества кандидатов увеличит и время предсказания модели, так как ей придется в каждой иттерации просчитывать бОльшее количество кандидатов.

4. Для решения поставленной задачи было создано и наполнено векторное пространство из объектов анонамизированного набора данных (base). Пространство поделено на 20 кластеров, что ускоряет и уточняет процесс поиска. Для каждого объекта из train были найдены 10 ближайших векторов при помощи меры FlatL2 (при большом количестве не хватало оперативной памяти при формировании дататсетов для обучения и тестирования ML модели).

5. На тренировочном датасете (train) получаем значение метрики **accuracy@5=63.114**, что является удовлетворительным результатом работы модели. Такой результат удалось достичь за счет удалениея не информативных признаков и стандартизации данных. При этом на тестовой выборке (validation) значение метрики **accuracy@5= 62.96** достигнуто без учета ранжирования ML моделью.

6. В качестве ML модели был выбран CatBoostClassifier и проведен подбор гиперпараметров. По обученной модели пердсказаны вероятности совместимости и по найденным занчениям вероятностей  определены 5 наиболее подходящих кандидато по которым расчитано значение целевой метрики **accuracy@5=61.81**

**На текущем этапе мы видим, что применение ML модели не улучшает показатель целевой метрики, темне-менне в дальнейшем можно поробовать усовршенстовать модель и повторно сравнить результат**